# NFL DATA CLEAN UP

Here we take the csv and clean up the data. We will be looking specifically at the Saints and their record over the past 5 seasons starting with 2010. The data we will be using was taken from http://www.footballdb.com/teams/nfl/new-orleans-saints/history. Check the other notebook for the webscraping program used to harness the data. 

In [1]:
%pylab inline
import pandas as pd
import re
# we have to clean up the NFL raw data
# import the dataset into a pandas data frame
df = pd.read_csv('datasets/saintsstats.csv' ,encoding='latin1')

Populating the interactive namespace from numpy and matplotlib


In [2]:
df[:3]

,Date,Opponent,Location,Result,Attend
0,09/09/2010 (Thu),Minnesota,"Louisiana Superdome (New Orleans, LA)",W 14-9,"70,051"
1,09/20/2010 (Mon),@ San Francisco,"Candlestick Park (San Francisco, CA)",W 25-22,"69,732"
2,9/26/10,Atlanta,"Louisiana Superdome (New Orleans, LA)","L 27-24, OT","70,051"


First we are going to clean the date column. Looking at the column we see that some entries have the day in parentheses. We need to remove these. After these are removed we are going to use the newly cleaned date column as the index for our database. 

In [3]:
df['Date']=df['Date'].str.replace(' \([^)]*\)','')
df['Date'][:3]

0    09/09/2010
1    09/20/2010
2       9/26/10
Name: Date, dtype: object

In [4]:
df=df.set_index(pd.DatetimeIndex(df['Date']))
df=df.sort_index()
df=df.drop('Date', 1)

df[:3]

,Opponent,Location,Result,Attend
2009-09-13,Detroit,"Louisiana Superdome (New Orleans, LA)",W 45-27,"69,719"
2009-09-20,@ Philadelphia,"Lincoln Financial Field (Philadelphia, PA)",W 48-22,"69,144"
2009-09-27,@ Buffalo,"Ralph Wilson Stadium (Orchard Park, NY)",W 27-7,"70,261"


Now our database is starting to look better. We have a few more things to clean up. Next we are going to fix up the opponent column. We are going to remove the @, however this causes us to lose some data. Instead of simply removing it we are going to create a new column that lists home or away. 


In [5]:
is_away=df['Opponent'].str[0]=='@'
advantage=[]
for i in range(0 , len(is_away)):
    if is_away[i] == True:
        advantage.append( 'Away')
    else:
        advantage.append('Home')
df['Playing'] = pd.Series(advantage, index=df.index)
df['Opponent'] = df['Opponent'].str.replace('@','')
df[:10]

,Opponent,Location,Result,Attend,Playing
2009-09-13,Detroit,"Louisiana Superdome (New Orleans, LA)",W 45-27,"69,719",Home
2009-09-20,Philadelphia,"Lincoln Financial Field (Philadelphia, PA)",W 48-22,"69,144",Away
2009-09-27,Buffalo,"Ralph Wilson Stadium (Orchard Park, NY)",W 27-7,"70,261",Away
2009-10-04,NY Jets,"Louisiana Superdome (New Orleans, LA)",W 24-10,"70,009",Home
2009-10-18,NY Giants,"Louisiana Superdome (New Orleans, LA)",W 48-27,"70,011",Home
2009-10-25,Miami,"Sun Life Stadium (Miami Gardens, FL)",W 46-34,"66,689",Away
2009-11-02,Atlanta,"Louisiana Superdome (New Orleans, LA)",W 35-27,"70,088",Home
2009-11-08,Carolina,"Louisiana Superdome (New Orleans, LA)",W 30-20,"70,011",Home
2009-11-15,St. Louis,"Edward Jones Dome (St. Louis, MO)",W 28-23,"59,858",Away
2009-11-22,Tampa Bay,"Raymond James Stadium (Tampa, FL)",W 38-7,"62,720",Away


Lets fix up the Location column next. I dont particularly care about the actual stadium name, but rather the city that it is in. Let's fix that.

In [6]:
df['Location']=df['Location'].str.split('(').str.get(1).str[:-1]
df[:10]

,Opponent,Location,Result,Attend,Playing
2009-09-13,Detroit,"New Orleans, LA",W 45-27,"69,719",Home
2009-09-20,Philadelphia,"Philadelphia, PA",W 48-22,"69,144",Away
2009-09-27,Buffalo,"Orchard Park, NY",W 27-7,"70,261",Away
2009-10-04,NY Jets,"New Orleans, LA",W 24-10,"70,009",Home
2009-10-18,NY Giants,"New Orleans, LA",W 48-27,"70,011",Home
2009-10-25,Miami,"Miami Gardens, FL",W 46-34,"66,689",Away
2009-11-02,Atlanta,"New Orleans, LA",W 35-27,"70,088",Home
2009-11-08,Carolina,"New Orleans, LA",W 30-20,"70,011",Home
2009-11-15,St. Louis,"St. Louis, MO",W 28-23,"59,858",Away
2009-11-22,Tampa Bay,"Tampa, FL",W 38-7,"62,720",Away


The last column we have to deal with is the Result column. This column should be split into three columns: Win/lose, Saints Score, and Opponent's Score.

In [7]:
OppScore = df['Result'].str.split('-').str.get(1)
SaintScore = df['Result'].str.split('-').str.get(0).str[2:]
df['O Score'] = pd.Series(OppScore, index=df.index)
df['H Score'] = pd.Series(SaintScore, index=df.index)
winLose = df['Result'].str[0]
df['Result'] = pd.Series(winLose, index=df.index)
df[:15]

,Opponent,Location,Result,Attend,Playing,O Score,H Score
2009-09-13,Detroit,"New Orleans, LA",W,"69,719",Home,27,45
2009-09-20,Philadelphia,"Philadelphia, PA",W,"69,144",Away,22,48
2009-09-27,Buffalo,"Orchard Park, NY",W,"70,261",Away,7,27
2009-10-04,NY Jets,"New Orleans, LA",W,"70,009",Home,10,24
2009-10-18,NY Giants,"New Orleans, LA",W,"70,011",Home,27,48
2009-10-25,Miami,"Miami Gardens, FL",W,"66,689",Away,34,46
2009-11-02,Atlanta,"New Orleans, LA",W,"70,088",Home,27,35
2009-11-08,Carolina,"New Orleans, LA",W,"70,011",Home,20,30
2009-11-15,St. Louis,"St. Louis, MO",W,"59,858",Away,23,28
2009-11-22,Tampa Bay,"Tampa, FL",W,"62,720",Away,7,38


There we have it a nice and clean database, but upon further inspection there is one more column we need to create. Some of the scores list overtime, so we need a column that tells us whether the game was won in overtime or not. We also need to strip the "OT" from the scores. 

In [8]:
is_OT=df['O Score'].str[-2:]=='OT'
OTs=[]
for i in range(0 , len(is_OT)):
    if is_OT[i] == True:
        OTs.append( 'True')
      
    else:
        OTs.append('False')
df['OT'] = pd.Series(OTs, index=df.index)
df['O Score'] = df['O Score'].str.replace('OT','')
df['O Score'] = df['O Score'].str.replace(',','')
df['O Score'] = df['O Score'].str.strip()
df[:15]

,Opponent,Location,Result,Attend,Playing,O Score,H Score,OT
2009-09-13,Detroit,"New Orleans, LA",W,"69,719",Home,27,45,False
2009-09-20,Philadelphia,"Philadelphia, PA",W,"69,144",Away,22,48,False
2009-09-27,Buffalo,"Orchard Park, NY",W,"70,261",Away,7,27,False
2009-10-04,NY Jets,"New Orleans, LA",W,"70,009",Home,10,24,False
2009-10-18,NY Giants,"New Orleans, LA",W,"70,011",Home,27,48,False
2009-10-25,Miami,"Miami Gardens, FL",W,"66,689",Away,34,46,False
2009-11-02,Atlanta,"New Orleans, LA",W,"70,088",Home,27,35,False
2009-11-08,Carolina,"New Orleans, LA",W,"70,011",Home,20,30,False
2009-11-15,St. Louis,"St. Louis, MO",W,"59,858",Away,23,28,False
2009-11-22,Tampa Bay,"Tampa, FL",W,"62,720",Away,7,38,False


Finally we save our dataset to a new csv.  Now we can do some fun visualizations, but first we need a little more data.

In [9]:
df.to_csv('datasets/saintsstatsclean.csv',encoding='latin1')